<a href="https://colab.research.google.com/github/Buitragox/Crowdsourcing-Thesis/blob/main/notebooks/get_majority_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pkl_path = "./drive/MyDrive/train_crowdsourced_labels.pkl"
df = pd.read_pickle(pkl_path)
df

,annotations,label,patch
0,"[[13, 1], [23, 1], [17, 2], [7, 1], [5, 1], [1...",1.0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...
1,"[[13, 1], [23, 1], [17, 2], [7, 1], [5, 1], [1...",1.0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...
2,"[[13, 1], [23, 1], [17, 2], [7, 1], [5, 1], [1...",1.0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...
3,"[[13, 1], [23, 1], [17, 2], [7, 1], [5, 1], [1...",1.0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...
4,"[[13, 1], [23, 1], [17, 1], [7, 1], [5, 1], [1...",1.0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...
...,...,...,...
75238,"[[5, 3]]",3.0,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...
75239,"[[5, 3]]",3.0,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...
75240,"[[5, 3]]",3.0,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...
75241,"[[5, 3]]",3.0,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...


In [ ]:
N = len(df)

image_ids = ["" for _ in range(N)]
mv = [0 for _ in range(N)]
labels = [0 for _ in range(N)]
amount_voters = [0 for _ in range(N)]
ties = {}
annotators = {}


for i in range(N):
    label_counts = [0, 0, 0]

    for annotation in df.iloc[i]["annotations"]:
        label = annotation[1] - 1
        label_counts[label] += 1

        annotator = annotation[0]
        if annotator in annotators:
            annotators[annotator] += 1
        else:
            annotators[annotator] = 1

    image_ids[i] = (df.iloc[i]["patch"])
    mv[i] = np.argmax(label_counts) + 1
    amount_voters[i] = np.sum(label_counts)
    labels[i] = int(df.iloc[i]["label"])

    # look for ties
    max_votes = np.max(label_counts)
    values = [(k+1, v) for k, v in enumerate(label_counts) if v == max_votes]
    if len(values) > 1:
        ties[df.iloc[i]["patch"]] = values

data = pd.DataFrame({"patch": image_ids, "mv": mv, "amount_voters": amount_voters, "label": labels})

In [ ]:
data

,patch,mv,amount_voters,label
0,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...,1,17,1
1,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...,1,17,1
2,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...,1,17,1
3,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...,1,17,1
4,TCGA-A1-A0SK-DX1_xmin47201_ymin23382_A0SK_A1_x...,1,17,1
...,...,...,...,...
75238,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...,3,1,3
75239,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...,3,1,3
75240,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...,3,1,3
75241,TCGA-S3-AA10-DX1_xmin43039_ymin23986_AA10_S3_x...,3,1,3


In [ ]:
data.to_pickle('./drive/MyDrive/mv.pkl') #save MV to pkl file

In [ ]:
print(f"Amount of ties in MV: {len(ties)} / {len(data)}")
list(ties.values())

Amount of ties in MV: 25 / 75243


[[(1, 8), (2, 8)],
 [(1, 4), (3, 4)],
 [(1, 8), (2, 8)],
 [(1, 9), (2, 9)],
 [(1, 8), (2, 8)],
 [(2, 9), (3, 9)],
 [(1, 8), (2, 8)],
 [(2, 7), (3, 7)],
 [(2, 8), (3, 8)],
 [(1, 9), (2, 9)],
 [(1, 7), (2, 7)],
 [(1, 7), (2, 7)],
 [(1, 6), (2, 6)],
 [(2, 5), (3, 5)],
 [(2, 4), (3, 4)],
 [(1, 9), (2, 9)],
 [(1, 9), (2, 9)],
 [(2, 7), (3, 7)],
 [(2, 8), (3, 8)],
 [(2, 8), (3, 8)],
 [(2, 5), (3, 5)],
 [(2, 4), (3, 4)],
 [(2, 4), (3, 4)],
 [(2, 8), (3, 8)],
 [(2, 9), (3, 9)]]

In [ ]:
counts = data["amount_voters"].value_counts()
counts_df = counts.to_frame(name="")
counts_df = counts_df.rename(columns={'': 'Cantidad de imagenes'})
counts_df.index.name = "Número de anotaciones"
counts_df

,Cantidad de imagenes
Número de anotaciones,
1,73103
19,615
18,345
17,297
16,254
11,188
20,110
15,105
14,60


In [ ]:
print("images:", len(data))
more_than_1 = (len(data) - 73103)
print(f"\nAmount of images with more than 1 voter: {more_than_1}")
print(f"Percentaje of images with more than 1 voter: {more_than_1 / len(data) * 100:.3f}%")

images: 75243

Amount of images with more than 1 voter: 2140
Percentaje of images with more than 1 voter: 2.844%


In [ ]:
print("Total amount of annotators:", len(annotators))
sorted(list(annotators.items()))

20


[(5, 5036),
 (6, 5728),
 (7, 6896),
 (8, 6084),
 (9, 5891),
 (10, 7930),
 (11, 4092),
 (12, 8581),
 (13, 5269),
 (14, 8223),
 (15, 6236),
 (16, 5518),
 (17, 2100),
 (18, 748),
 (19, 5739),
 (20, 3980),
 (21, 4816),
 (22, 2285),
 (23, 5481),
 (24, 7862)]

In [ ]:
print("Average amount of annotated samples per annotator:", np.mean(list(annotators.values())))

Average amount of annotated samples per annotator: 5424.75


In [ ]:
print("Class distribution for Gold Standard\n=============================")

label_counts = list(data["label"].value_counts().items())
print(f"Images per class: {label_counts}")

for k, n in label_counts:
    print(f"{k} {n / len(data) * 100:.4f}%")

Class distribution for Gold Standard
Images per class: [(1, 37260), (2, 27668), (3, 10315)]
1 49.5196%
2 36.7715%
3 13.7089%


In [ ]:
print("Class distribution for Majority Voting\n=============================")

label_counts = list(data["mv"].value_counts().items())
print("Entire dataset")
print(f"Images per class: {label_counts}")

for k, n in label_counts:
    print(f"k {n / len(data):.4f}%")

print("\nMore than 1 annotator")
label_counts = list(data[data["amount_voters"]>1]["mv"].value_counts().items())
total = sum(map(lambda x: x[1], label_counts))
print(label_counts)
for k, n in label_counts:
    print(f"{k} {n / total * 100:.4f}%")

Class distribution for Majority Voting
Entire dataset
Images per class: [(1, 36217), (2, 28955), (3, 10071)]
k 0.4813%
k 0.3848%
k 0.1338%

More than 1 annotator
[(1, 963), (2, 822), (3, 355)]
1 45.0000%
2 38.4112%
3 16.5888%


In [ ]:
amount_equal = len(data[data["mv"]==data["label"]])
print(f"Images: {N}")
print(f"Images where MV == true label: {amount_equal} | {amount_equal / N * 100 :.3f}%")

multiple_ann = data[data["amount_voters"]>1]
amount_equal_multiple_ann = len(multiple_ann[multiple_ann["mv"]==multiple_ann["label"]])

print(f"\nImages with multiple annotators: {more_than_1}")
print(f"Images with multiple annotators where MV == true label: {amount_equal_multiple_ann} | {amount_equal_multiple_ann / more_than_1 * 100 :.3f}%")

Images: 75243
Images where MV == true label: 69718 | 92.657%

Images with multiple annotators: 2140
Images with multiple annotators where MV == true label: 1851 | 86.495%


# LaTeX Tables

This section is used for printing latex tables to be used in the final document

In [ ]:
print(pd.DataFrame(annotators.items(), columns=["Annotator ID", "Votes"]).sort_values("Annotator ID", ascending=True).to_latex(index=False))

\begin{tabular}{rr}
\toprule
Annotator ID & Votes \\
\midrule
5 & 5036 \\
6 & 5728 \\
7 & 6896 \\
8 & 6084 \\
9 & 5891 \\
10 & 7930 \\
11 & 4092 \\
12 & 8581 \\
13 & 5269 \\
14 & 8223 \\
15 & 6236 \\
16 & 5518 \\
17 & 2100 \\
18 & 748 \\
19 & 5739 \\
20 & 3980 \\
21 & 4816 \\
22 & 2285 \\
23 & 5481 \\
24 & 7862 \\
\bottomrule
\end{tabular}



In [ ]:
print(counts_df.to_latex())

\begin{tabular}{lr}
\toprule
 & Cantidad de imagenes \\
Número de anotaciones &  \\
\midrule
1 & 73103 \\
19 & 615 \\
18 & 345 \\
17 & 297 \\
16 & 254 \\
11 & 188 \\
20 & 110 \\
15 & 105 \\
14 & 60 \\
10 & 51 \\
13 & 35 \\
9 & 30 \\
12 & 21 \\
8 & 15 \\
7 & 10 \\
6 & 4 \\
\bottomrule
\end{tabular}

